In [1]:
import urllib.request
import json
from pulp import *
import numpy as np

In [2]:
# download current season data

with urllib.request.urlopen("https://fantasy.premierleague.com/api/bootstrap-static/") as url:
    data = json.loads(url.read().decode())

# Start of season picking

##  find optimal full squad for a previous season with current players and current price  

In [3]:
# just assume that you get points from all 15 players in team throughout season
# constraints include price as well as 3 players from same team

# this finds optimal team looking at the statistic you want to maximise - eg points, points per game, creativity etc

In [130]:
# pick season to look at
season = '2018/19'

''' pick statistic to optimise. Can be one of the following
{'assists',
  'bonus',
  'bps',
  'creativity',
  'goals_scored',
  'ict_index',
  'influence',
  'minutes',
  'threat',
  'total_points'
}
'''

stat = 'total_points'

pos = {1: 'goalkeeper', 2: 'defender', 3: 'midfielder', 4: 'forward'}

# get dictionary of player name and [corresponding id, start_of_season_cost, positon, club]
player_id_dict = {x['web_name']: [x['id'],
                                  x['now_cost'] - x['cost_change_start'],
                                  pos[x['element_type']],
                                  x['team']]
                  for x in data['elements']}

In [131]:
# for each player create dict with name: [statistic, start_of_season_cost, postion, club]
# if not played then statistic is 0

last_season_dict = {}

for key in player_id_dict.keys():
    # download player specific data
    with urllib.request.urlopen("https://fantasy.premierleague.com/api/element-summary/{}/" 
                                .format(player_id_dict[key][0])) as url:
        data_player = json.loads(url.read().decode())

    for x in data_player['history_past']:
        if x['season_name'] == season:
            last_season_dict[key] = [float(
                x[stat]), player_id_dict[key][1], player_id_dict[key][2], player_id_dict[key][3]]

In [132]:
# maximise statistic subject to constraints

player = list(last_season_dict.keys())
stat = {key: last_season_dict[key][0] for key in last_season_dict.keys()}
cost = {key: last_season_dict[key][1] for key in last_season_dict.keys()}
gk = {key: 1 if last_season_dict[key][2] ==
      'goalkeeper' else 0 for key in last_season_dict.keys()}
defe = {key: 1 if last_season_dict[key][2] ==
        'defender' else 0 for key in last_season_dict.keys()}
mid = {key: 1 if last_season_dict[key][2] ==
       'midfielder' else 0 for key in last_season_dict.keys()}
stri = {key: 1 if last_season_dict[key][2] ==
        'forward' else 0 for key in last_season_dict.keys()}

prob = LpProblem("Fantasy Football", LpMaximize)
player_vars = LpVariable.dicts("Player", player, 0, 1, LpBinary)

# objective function
prob += lpSum([stat[i]*player_vars[i] for i in player]), "Total Statistic"

# constraint
prob += lpSum([player_vars[i] for i in player]) == 15, "Total 15 Players"
prob += lpSum([cost[i] * player_vars[i]
               for i in player]) <= 1000.0, "Total Cost"
prob += lpSum([gk[i] * player_vars[i] for i in player]) == 2, "Only 1 GK"
prob += lpSum([defe[i] * player_vars[i] for i in player]) == 5, "5 DEF"
prob += lpSum([mid[i] * player_vars[i] for i in player]) == 5, "5 MID"
prob += lpSum([stri[i] * player_vars[i] for i in player]) == 3, "3 STR"

# add constraint of max 3 in a team
for i in range(20):  # 20 teams in the league
    club = {key: 1 if last_season_dict[key][3] ==
            i else 0 for key in last_season_dict.keys()}
    prob += lpSum([club[i] * player_vars[i] for i in player]
                  ) <= 3, "Less than 3 in team {}".format(i)

# solve
status = prob.solve()

In [135]:
#print('total points are {} \n'.format(value(prob.objective)))
print('players in team are: ')

for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

players in team are: 
Player_Alisson = 1.0
Player_Azpilicueta = 1.0
Player_Callum_Wilson = 1.0
Player_David_Luiz = 1.0
Player_Felipe_Anderson = 1.0
Player_Fraser = 1.0
Player_Jiménez = 1.0
Player_Jota = 1.0
Player_Kanté = 1.0
Player_Laporte = 1.0
Player_Milivojevic = 1.0
Player_Pickford = 1.0
Player_Robertson = 1.0
Player_Sigurdsson = 1.0
Player_van_Dijk = 1.0


## find optimal starting team for last season with current players and current price

In [136]:
# assume no transfers throughout entire season so want to maximise points in starting line up.
# constraints include price as well as 3 players from same team

# so you buy 15 players
# 2 goalkeepers, 5 defenders, 5 midfielders and 3 forwards
# you have 4 on bench (1 gk, 3 remaining players). Want to maximise points in 11 playing

In [137]:
# possible formations are 3-4-3, 3-5-2, 4-3-3, 4-4-2, 4-5-1, 5-2-3, 5-3-2 and 5-4-1
# for each formation I make the cheapest bench possible. I then optimise stat at price (1000 - price_of_bench)
# I then pick the formation with starting team that gets the most points

In [138]:
# find minimum gk, def, mid and stri values
position_stat_dict = {}

for i in range(4):
    prices_pos = [last_season_dict[key][1] for key in last_season_dict.keys()
                  if list(pos.values()).index(last_season_dict[key][2]) == i]

    position_stat_dict[i] = min(prices_pos)

In [140]:
formations = {0: [3, 4, 3], 1: [3, 5, 2], 2: [4, 3, 3], 3: [4, 4, 2],
              4: [4, 5, 1], 5: [5, 2, 3], 6: [5, 3, 2], 7: [5, 4, 1]}


team_stat_dict = {}

values = []


for key in formations.keys():
    bench_price = position_stat_dict[0] + (5 - formations[key][0])*position_stat_dict[1] + (
        5 - formations[key][1])*position_stat_dict[2] + (3 - formations[key][2])*position_stat_dict[3]

    prob = LpProblem("Fantasy Football", LpMaximize)
    player_vars = LpVariable.dicts("Player", player, 0, 1, LpBinary)

    # objective function
    prob += lpSum([stat[i]*player_vars[i] for i in player]), "Total Statistic"

    # constraint
    prob += lpSum([player_vars[i] for i in player]) == 11, "Total 11 Players"
    prob += lpSum([cost[i] * player_vars[i] for i in player]
                  ) <= (1000.0 - bench_price), "Total Cost"
    prob += lpSum([gk[i] * player_vars[i] for i in player]) == 1, "Only 1 GK"
    prob += lpSum([defe[i] * player_vars[i]
                   for i in player]) == formations[key][0], "DEF"
    prob += lpSum([mid[i] * player_vars[i]
                   for i in player]) == formations[key][1], "MID"
    prob += lpSum([stri[i] * player_vars[i]
                   for i in player]) == formations[key][2], "STR"

    # add constraint of max 3 in a team
    for i in range(20):  # 20 teams in the league
        club = {key: 1 if last_season_dict[key][3] ==
                i else 0 for key in last_season_dict.keys()}
        prob += lpSum([club[i] * player_vars[i] for i in player]
                      ) <= 3, "Less than 3 in team {}".format(i)

    # solve
    status = prob.solve()

    players = [v.name for v in prob.variables() if v.varValue > 0]
    dicti = {str(key):value for key, value in zip(list(prob.objective.keys()), list(prob.objective.values()))}
    val = 0
    for play in players:
        val += dicti[play]
    values.append(val)
    team_stat_dict[key] = [val, players]

pos_of_max = np.argmax(values)
print('best formation is {} \n'.format(formations[pos_of_max]))
print('total points are {} \n'.format(team_stat_dict[pos_of_max][0]))
print('players in team are: {}'.format(team_stat_dict[pos_of_max][1]))

best formation is [5, 4, 1] 

total points are 2032.0 

players in team are: ['Player_David_Luiz', 'Player_Fraser', 'Player_Jiménez', 'Player_Laporte', 'Player_Milivojevic', 'Player_Pickford', 'Player_Robertson', 'Player_Salah', 'Player_Sigurdsson', 'Player_van_Aanholt', 'Player_van_Dijk']


## optimal player recommendation from last season

In [ ]:
# you specify positions in team remaining eg [goalkeeper, defender, defender], price left and statistic you want to maximise and
# this will recommend the players that last season would maximise this



# During season transfers

## in-form best 11 with constrained price 

In [27]:
# you specify num of previous gameweeks to look at, constrained price and statistic to optimise
# and this shows best starting team over those gameweeks

In [124]:
num_prev_matches = 5
statistic = 'total_points'
constrained_price = 970

# get dict of the form player_name:[total_points_over_last_n_matches, current_cost, position, club]

#player_id_dict has form player name: [corresponding id, start_of_season_cost, positon, club]

In [125]:
player_form_dict = {}

for key in player_id_dict.keys():
    # download player specific data
    with urllib.request.urlopen("https://fantasy.premierleague.com/api/element-summary/{}/" 
                                .format(player_id_dict[key][0])) as url:
        data_player = json.loads(url.read().decode())
        
    statistic_total = 0.0
    value = data_player['history'][-1]['value']
        
    for idx, x in enumerate(reversed(data_player['history'])):
        if idx < num_prev_matches:
            statistic_total += float(x[statistic])
            
        else:
            break
            
    player_form_dict[key] = [statistic_total, value, player_id_dict[key][2], player_id_dict[key][3]]

In [126]:
player = list(player_form_dict.keys())
stat = {key: player_form_dict[key][0] for key in player_form_dict.keys()}
cost = {key: player_form_dict[key][1] for key in player_form_dict.keys()}
gk = {key: 1 if player_form_dict[key][2] ==
      'goalkeeper' else 0 for key in player_form_dict.keys()}
defe = {key: 1 if player_form_dict[key][2] ==
        'defender' else 0 for key in player_form_dict.keys()}
mid = {key: 1 if player_form_dict[key][2] ==
       'midfielder' else 0 for key in player_form_dict.keys()}
stri = {key: 1 if player_form_dict[key][2] ==
        'forward' else 0 for key in player_form_dict.keys()}

prob = LpProblem("Fantasy Football", LpMaximize)
player_vars = LpVariable.dicts("Player", player, 0, 1, LpBinary)

# objective function
prob += lpSum([stat[i]*player_vars[i] for i in player]), "Total Statistic"

# constraint
prob += lpSum([player_vars[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_vars[i]
               for i in player]) <= constrained_price, "Total Cost"
prob += lpSum([gk[i] * player_vars[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defe[i] * player_vars[i] for i in player]) <= 5, "less than 5 DEF"
prob += lpSum([defe[i] * player_vars[i] for i in player]) >= 3, "more than 3 DEF"
prob += lpSum([mid[i] * player_vars[i] for i in player]) <= 5, "less than 5 MID"
prob += lpSum([mid[i] * player_vars[i] for i in player]) >= 2, "more than 2 MID"
prob += lpSum([stri[i] * player_vars[i] for i in player]) <= 3, "less than 3 STR"
prob += lpSum([stri[i] * player_vars[i] for i in player]) >= 1, "more than 1 STR"

# add constraint of max 3 in a team
for i in range(20):  # 20 teams in the league
    club = {key: 1 if player_form_dict[key][3] ==
            i else 0 for key in player_form_dict.keys()}
    prob += lpSum([club[i] * player_vars[i] for i in player]
                  ) <= 3, "Less than 3 in team {}".format(i)

# solve
status = prob.solve()

In [127]:
print('players in team are: ')

for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

players in team are: 
Player_Abraham = 1.0
Player_Agüero = 1.0
Player_Alexander_Arnold = 1.0
Player_De_Bruyne = 1.0
Player_Digne = 1.0
Player_Mané = 1.0
Player_Pukki = 1.0
Player_Salah = 1.0
Player_Sterling = 1.0
Player_Vestergaard = 1.0
Player_de_Gea = 1.0


## in form best 11 constrained by bench

In [128]:
for key in formations.keys():
    bench_price = position_stat_dict[0] + (5 - formations[key][0])*position_stat_dict[1] + (
        5 - formations[key][1])*position_stat_dict[2] + (3 - formations[key][2])*position_stat_dict[3]

    prob = LpProblem("Fantasy Football", LpMaximize)
    player_vars = LpVariable.dicts("Player", player, 0, 1, LpBinary)

    # objective function
    prob += lpSum([stat[i]*player_vars[i] for i in player]), "Total Statistic"

    # constraint
    prob += lpSum([player_vars[i] for i in player]) == 11, "Total 11 Players"
    prob += lpSum([cost[i] * player_vars[i] for i in player]
                  ) <= (constrained_price - bench_price), "Total Cost"
    prob += lpSum([gk[i] * player_vars[i] for i in player]) == 1, "Only 1 GK"
    prob += lpSum([defe[i] * player_vars[i]
                   for i in player]) == formations[key][0], "DEF"
    prob += lpSum([mid[i] * player_vars[i]
                   for i in player]) == formations[key][1], "MID"
    prob += lpSum([stri[i] * player_vars[i]
                   for i in player]) == formations[key][2], "STR"

    # add constraint of max 3 in a team
    for i in range(20):  # 20 teams in the league
        club = {key: 1 if player_form_dict[key][3] ==
                i else 0 for key in player_form_dict.keys()}
        prob += lpSum([club[i] * player_vars[i] for i in player]
                      ) <= 3, "Less than 3 in team {}".format(i)

    # solve
    status = prob.solve()

    players = [v.name for v in prob.variables() if v.varValue > 0]
    dicti = {str(key):value for key, value in zip(list(prob.objective.keys()), list(prob.objective.values()))}
    val = 0
    for play in players:
        val += dicti[play]
    values.append(val)
    team_stat_dict[key] = [val, players]

pos_of_max = np.argmax(values)
print('best formation is {} \n'.format(formations[pos_of_max]))
print('total points are {} \n'.format(team_stat_dict[pos_of_max][0]))
print('players in team are: {}'.format(team_stat_dict[pos_of_max][1]))

best formation is [4, 5, 1] 

total points are 348.0 

players in team are: ['Player_Alexander_Arnold', 'Player_Cantwell', 'Player_De_Bruyne', 'Player_Digne', 'Player_Gunn', 'Player_Mané', 'Player_Mings', 'Player_Mount', 'Player_Pukki', 'Player_Salah', 'Player_Vestergaard']


# Data Visualisation suite